# Homework 3
W261
Chris Dailey

christopher.dailey@gmail.com

## HW 3.1
Use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories

In [4]:
!head Consumer_Complaints.csv -n 30

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1115890,Credit reporting,,Incorrect information on credit report,Information is

In [8]:
!hdfs dfs -mkdir /user/ubuntu/HW3
!hdfs dfs -mkdir /user/ubuntu/HW3/3.1

In [6]:
!hdfs dfs -put Consumer_Complaints.csv /user/ubuntu/HW3/

In [8]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    splits = re.split(",", line)
    if splits[1].lower() == "debt collection":
        sys.stderr.write("reporter:counter:Type,DebtCollection,1\n")
    elif splits[1].lower() == "mortgage":
        sys.stderr.write("reporter:counter:Type,Mortgage,1\n")
    else:
        sys.stderr.write("reporter:counter:Type,Other,1\n")

Overwriting mapper.py


In [9]:
!chmod +x mapper.py

In [10]:
!hdfs dfs -rm /user/ubuntu/HW3/3.1/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.1/output
!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.1/output

16/01/30 02:00:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.1/output/_SUCCESS
16/01/30 02:00:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.1/output/part-00000
packageJobJar: [/tmp/hadoop-unjar2402735397871742075/] [] /tmp/streamjob5694268014097702284.jar tmpDir=null
16/01/30 02:00:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:00:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:00:37 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:00:37 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:00:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0023
16/01/30 02:00:37 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0023
16/01/30 02:00

## HW 3.2
Perform a word count analysis of the phrase "foo foo quux labs foo bar quux" and count the number of mappers and reducers and explain the results.

In [50]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

for line in sys.stdin:
    for word in [x.strip().lower() for x in re.split(" ", line)]:
        print(word + ", " + str(1))

Overwriting mapper.py


In [24]:
!echo "test test test test" | python mapper.py

reporter:counter:MapperCount,1
test, 1
test, 1
test, 1
test, 1


In [49]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + " " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + " " + str(current_sum))

Overwriting reducer.py


In [39]:
!chmod +x reducer.py

In [40]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort | python reducer.py

reporter:counter:ReducerCount,1
reporter:counter:MapperCount,1
bar 1
foo 3
labs 1
quux 2


In [41]:
!hdfs dfs -mkdir /user/ubuntu/HW3/3.2

In [46]:
!echo "foo foo quux labs foo bar quux" > oneline.txt ; cat oneline.txt

foo foo quux labs foo bar quux


In [47]:
!hdfs dfs -put oneline.txt /user/ubuntu/HW3/3.2

In [52]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.2/oneline.txt \
-output /user/ubuntu/HW3/3.2/output

16/01/29 00:50:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/29 00:50:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar519971968270769378/] [] /tmp/streamjob435509402778434573.jar tmpDir=null
16/01/29 00:50:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/29 00:50:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/29 00:50:08 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/29 00:50:08 INFO mapreduce.JobSubmitter: number of splits:2
16/01/29 00:50:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0021
16/01/29 00:50:09 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0021
16/01/29 00:50:0

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called

In [11]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    issues = splits[3]
    for word in WORD_RE.findall(issues):
        print(word+", 1")

Overwriting mapper.py


In [3]:
!echo ["","","","test this is a test"] | python mapper.py

reporter:counter:TaskCounter,Mapper,1
test, 1
this, 1
is, 1
a, 1
test, 1


In [12]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + " " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + " " + str(current_sum))

Overwriting reducer.py


In [13]:
!echo ["","","","test this is a test"] | python mapper.py | sort | python reducer.py

reporter:counter:TaskCounter,Mapper,1
reporter:counter:TaskCounter,Reducer,1
a 1
is 1
test 2
this 1


In [14]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/30 02:02:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/30 02:02:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar5712215938276641105/] [] /tmp/streamjob6344048016643955809.jar tmpDir=null
16/01/30 02:02:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:02:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:02:18 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:02:18 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:02:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0024
16/01/30 02:02:18 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0024
16/01/30 02:02

The TaskCounter reads:<br>
		Mapper=2<br>
		Reducer=1
    

In [15]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2/output/part-00000

a 3503	
account 20681	
acct 163	
action 2505	
advance 240	
advertising 1193	
amount 98	
amt 71	
an 2505	
and 16448	
application 8868	
applied 139	
apply 118	
apr 3431	
arbitration 168	
are 3821	
atm 2422	
attempts 11848	
available 274	
balance 597	
bank 202	
bankruptcy 222	
being 5663	
billing 8158	
by 5663	
can't 1999	
cancelling 2795	
card 4405	
cash 240	
caused 5663	
changes 350	
charged 976	
charges 131	
checks 75	
closing 2795	
club 12545	
collect 11848	
collection 1907	
communication 6920	
company's 4858	
cont'd 11848	
contact 3053	
convenience 75	
costs 4350	
credit 55251	
credited 92	
customer 2734	
day 71	
dealing 1944	
debit 2422	
debt 19309	
decision 2774	
decrease 1149	
delay 243	
delinquent 1061	
deposits 10555	
determination 1490	
did 139	
didn't 925	
disclosure 5214	
disclosures 64	
dispute 904	
disputes 6938	
embezzlement 3276	
expect 807	
false 2508	
fee 3198	
fees 807	
for 929	
forbearance 350	
fraud 3842	
funds 5663	
get 4357	
getting 291	
health 12545	
i 925	
identi

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [16]:
%%writefile combiner.py
#!/usr/bin/python

import sys, re

#this is the exact same as the reducer except the output is modified
#to include a comma between the key and value to fulfill the contract 
#that the combiner accept and emit keys in the same format as the mapper

sys.stderr.write("reporter:counter:TaskCounter,Combiner,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + ", " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + ", " + str(current_sum))

Writing combiner.py


In [17]:
!chmod +x combiner.py

In [18]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-combiner /home/ubuntu/repos/261/HW3/combiner.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/30 02:54:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/30 02:54:26 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar1646522515064830425/] [] /tmp/streamjob1924805495545833480.jar tmpDir=null
16/01/30 02:54:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:54:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:54:30 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:54:30 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:54:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0025
16/01/30 02:54:30 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0025
16/01/30 02:54

Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).

In [19]:
#Build an order-inverting mapper

In [64]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

marginal = 0

for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    issues = splits[3]
    for word in WORD_RE.findall(issues):
        print(word+"\t1")
        marginal += 1
print("***\t" + str(marginal))

Overwriting mapper.py


In [65]:
#Build an inversion-aware reducer

In [71]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

marginal = 0

current_word = "***"
current_sum = 0
for line in sys.stdin:  
    splits = [x.lower().strip() for x in re.split("\t", line)]
    if (len(splits)) < 2:
        continue
    if splits[0] != current_word:
        if current_word != "***":
            print(current_word + "\t" + str(current_sum) + "\t" + str(1.*current_sum/marginal))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        if current_word == "***":
            marginal += int(splits[1])
        else:
            current_sum += int(splits[1])
print(current_word + "\t" + str(current_sum) + "\t" + str(1.*current_sum/marginal))

Overwriting reducer.py


In [72]:
!cat Consumer_Complaints_mini.csv | python mapper.py | sort | python reducer.py

reporter:counter:TaskCounter,Mapper,1
reporter:counter:TaskCounter,Reducer,1
a	2	0.0009474182851729038
account	19	0.009000473709142587
action	3	0.0014211274277593558
advertising	1	0.0004737091425864519
an	3	0.0014211274277593558
and	18	0.008526764566556135
application	3	0.0014211274277593558
applied	1	0.0004737091425864519
apr	3	0.0014211274277593558
are	3	0.0014211274277593558
atm	2	0.0009474182851729038
attempts	37	0.01752723827569872
available	1	0.0004737091425864519
balance	1	0.0004737091425864519
being	9	0.004263382283278067
billing	1	0.0004737091425864519
by	9	0.004263382283278067
cancelling	1	0.0004737091425864519
can't	3	0.0014211274277593558
card	2	0.0009474182851729038
caused	9	0.004263382283278067
charged	2	0.0009474182851729038
closing	1	0.0004737091425864519
club	55	0.026054002842254856
collect	37	0.01752723827569872
communication	24	0.011369019422074847
company's	40	0.018948365703458078
contact	9	0.004263382283278067
cont'd	37	0.01752723827569872
costs	4	0.001894836570345

In [73]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/31 16:05:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/31 16:05:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar7543454808455630869/] [] /tmp/streamjob4492476671369596750.jar tmpDir=null
16/01/31 16:05:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:05:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:05:27 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 16:05:27 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 16:05:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0051
16/01/31 16:05:28 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0051
16/01/31 16:05

In [74]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2/output/part-00000

a	3503	0.00357272895094
account	20681	0.0210926655536
acct	163	0.000166244595776
action	2505	0.00255486326637
advance	240	0.000244777318934
advertising	1193	0.0012167472562
amount	98	9.99507385646e-05
amt	71	7.24132901845e-05
an	2505	0.00255486326637
and	16448	0.0167754055909
application	8868	0.0090445219346
applied	139	0.000141766863882
apply	118	0.000120348848476
apr	3431	0.00349929575526
arbitration	168	0.000171344123254
are	3821	0.00389705889852
atm	2422	0.00247021111024
attempts	11848	0.0120838403114
available	274	0.000279454105783
balance	597	0.000608883580847
bank	202	0.000206020910102
bankruptcy	222	0.000226419020014
being	5663	0.00577572482134
billing	8158	0.00832038903275
by	5663	0.00577572482134
can't	1999	0.00203879108562
cancelling	2795	0.00285063586008
card	4405	0.00449268370793
cash	240	0.000244777318934
caused	5663	0.00577572482134
changes	350	0.000356966923445
charged	976	0.000995427763663
charges	131	0.000133607619918
checks	75	7.64929121668e-05
closing	2795	0.0028506

In [1]:
%%writefile mapper.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    print(line.strip())

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/python

import sys,re

#ount = 0

for line in sys.stdin:
    print(line.strip())
    #ount += 1
    #f count >= 50:
    #   break

Overwriting reducer.py


In [17]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2.4/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2.4/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.num.map.output.key.field=3 \
-D stream.map.output.field.separator="\t" \
-D mapreduce.partition.keypartitioner.options=-k2,2 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr" \
-D mapred.reduce.tasks=2 \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.2/output/part-00000 \
-output /user/ubuntu/HW3/3.2.4/output

16/02/02 14:02:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2.4/output/_SUCCESS
16/02/02 14:02:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2.4/output/part-00000
16/02/02 14:02:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2.4/output/part-00001
packageJobJar: [/tmp/hadoop-unjar3700287971485110681/] [] /tmp/streamjob2166345673225387406.jar tmpDir=null
16/02/02 14:02:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/02 14:02:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/02 14:02:12 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 14:02:12 INFO mapreduce.JobSubmitter: number of splits:2
16/02/02 14:02:12 

In [18]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2.4/output/part-0000*

loan	119630	0.122011294433
modification	70487	0.0718900786653
credit	55251	0.056350798535
servicing	36767	0.0374988653551
report	34903	0.0355977615114
incorrect	29133	0.0297129068021
information	29069	0.0296476328503
on	29069	0.0296476328503
or	22533	0.0229815305314
account	20681	0.0210926655536
debt	19309	0.0196933552137
and	16448	0.0167754055909
opening	16205	0.0165275685555
health	12545	0.0127947144418
club	12545	0.0127947144418
not	12353	0.0125988925866
owed	11848	0.0120838403114
cont'd	11848	0.0120838403114
attempts	11848	0.0120838403114
collect	11848	0.0120838403114
of	10885	0.0111016713191
my	10731	0.0109446058728
withdrawals	10555	0.0107651025056
deposits	10555	0.0107651025056
problems	9484	0.00967278371986
application	8868	0.0090445219346
to	8401	0.00856822606817
unable	8178	0.00834078714266
billing	8158	0.00832038903275
other	7886	0.00804297473796
disputes	6938	0.00707610432817
communication	6920	0.00705774602925
tactics	6920	0.00705774602925
reporting	6559	0.00668956014536
l

## HW 3.3
Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.

In [86]:
!hdfs dfs -mkdir /user/ubuntu/HW3/3.3

In [87]:
!hdfs dfs -put ./ProductPurchaseData.txt /user/ubuntu/HW3/3.3/

### Calculate number of unique products

In [98]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

products = set()

for line in sys.stdin:
    for product in [x.strip() for x in re.split(" ", line)]:
        if product not in products:
            products.add(product)
            #send only one k,v from a mapper.  The reducer is guaranteed to get
            #all the products with the same key so even if there are multiple mappers
            #the reducers will be able to deduplicate the keys they get
            print(product + "\t1")

Overwriting mapper.py


In [94]:
!head ProductPurchaseData.txt -n50 >> ProductPurchaseDataMini.txt

In [99]:
!cat ProductPurchaseDataMini.txt | python mapper.py

FRO11987	1
ELE17451	1
ELE89019	1
SNA90258	1
GRO99222	1
	1
GRO12298	1
FRO12685	1
ELE91550	1
SNA11465	1
ELE26917	1
ELE52966	1
FRO90334	1
SNA30755	1
FRO84225	1
SNA80192	1
GRO73461	1
DAI22896	1
SNA99873	1
FRO86643	1
ELE37798	1
GRO56989	1
ELE23393	1
SNA69641	1
FRO78087	1
GRO39357	1
ELE28573	1
ELE11375	1
DAI54444	1
FRO18919	1
DAI50921	1
GRO75578	1
ELE59935	1
SNA85662	1
SNA91554	1
DAI22177	1
DAI14125	1
DAI46755	1
FRO81176	1
ELE66810	1
DAI49199	1
DAI91535	1
GRO94758	1
ELE94711	1
FRO76833	1
DAI54690	1
SNA47306	1
GRO36567	1
ELE82555	1
SNA17715	1
SNA94781	1
DAI87514	1
GRO48282	1
GRO12935	1
SNA55952	1
DAI93692	1
DAI92253	1
FRO82427	1
DAI48891	1
ELE27376	1
FRO47475	1
DAI17810	1
FRO17376	1
DAI38969	1
DAI50913	1
ELE11111	1
SNA66979	1
FRO92261	1
FRO76487	1
GRO39369	1
GRO22575	1
GRO48971	1
FRO32293	1
FRO98729	1
DAI95741	1
DAI44355	1
FRO75418	1
FRO70974	1
GRO49037	1
GRO30386	1
FRO41069	1
ELE96863	1
FRO16142	1
ELE52446	1
DAI88808	1
FRO36081	1
DAI35347	1
SNA63157	1
ELE73246	1
ELE66067	1
DAI22534	1
SNA9386

In [100]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re


#Start counting at one.  This will produce an inaccurate reading of 1 product if there
#are actually 0, but I don't think we need to worry about that case
product_count = 1

current_product = None

for line in sys.stdin:
    splits = re.split("\t", line)
    if current_product is None:
        current_product = splits[0]
    if splits[0] != current_product:
        product_count += 1
        
print(str(product_count))

Overwriting reducer.py


In [101]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort | python reducer.py

219


In [102]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.3/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.3/output

rm: `/user/ubuntu/HW3/3.3/output/*': No such file or directory
packageJobJar: [/tmp/hadoop-unjar1043300641705263494/] [] /tmp/streamjob6934915716189978039.jar tmpDir=null
16/01/31 18:22:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:22:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:22:20 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 18:22:20 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 18:22:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0057
16/01/31 18:22:21 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0057
16/01/31 18:22:21 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0057/
16/01/31 18:22:21 INFO mapreduce.Job: Running job: job_1453815382601_0057
16/01/31 18:22:26 INFO mapreduce.Job: Job job_1453815382601_0057 running in uber mode : false
16/01/31 18:22:26 

In [104]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3/output/part-00000

17746	


### Find the largest basket

In [131]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

#a simple in-memory combiner
WORD_RE = re.compile(r"[\w']+")

largest_basket_so_far = 0

for line in sys.stdin:
    if len(WORD_RE.findall(line)) > largest_basket_so_far:
        largest_basket_so_far = len(WORD_RE.findall(line))
print(str(largest_basket_so_far))

Overwriting mapper.py


In [132]:
!cat ProductPurchaseDataMini.txt | python mapper.py

23


In [133]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    print(line.strip())
    break

Overwriting reducer.py


In [134]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort -k1nr | python reducer.py

23


In [135]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.3/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k1nr \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.3/output

16/01/31 19:18:19 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.3/output/_SUCCESS
16/01/31 19:18:19 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.3/output/part-00000
packageJobJar: [/tmp/hadoop-unjar116152308454992242/] [] /tmp/streamjob1892306868453652829.jar tmpDir=null
16/01/31 19:18:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 19:18:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 19:18:24 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 19:18:24 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 19:18:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0060
16/01/31 19:18:24 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0060
16/01/31 19:18:

In [136]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3/output/part-00000

37	


### Top 50 most frequently purchased items
Their frequency,  and their relative frequency (break ties by sorting the products alphabetical order)

In [157]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

total_purchases = 0

for line in sys.stdin:
    for product in [x.rstrip() for x in WORD_RE.findall(line)]:
        print(product + "\t1")
        total_purchases += 1
print("***\t" + str(total_purchases))

Overwriting mapper.py


In [158]:
!cat ProductPurchaseDataMini.txt | python mapper.py

FRO11987	1
ELE17451	1
ELE89019	1
SNA90258	1
GRO99222	1
GRO99222	1
GRO12298	1
FRO12685	1
ELE91550	1
SNA11465	1
ELE26917	1
ELE52966	1
FRO90334	1
SNA30755	1
ELE17451	1
FRO84225	1
SNA80192	1
ELE17451	1
GRO73461	1
DAI22896	1
SNA99873	1
FRO86643	1
ELE17451	1
ELE37798	1
FRO86643	1
GRO56989	1
ELE23393	1
SNA11465	1
ELE17451	1
SNA69641	1
FRO86643	1
FRO78087	1
SNA11465	1
GRO39357	1
ELE28573	1
ELE11375	1
DAI54444	1
ELE17451	1
GRO73461	1
DAI22896	1
SNA99873	1
FRO18919	1
DAI50921	1
SNA80192	1
GRO75578	1
ELE17451	1
ELE59935	1
FRO18919	1
ELE23393	1
SNA80192	1
SNA85662	1
SNA91554	1
DAI22177	1
ELE17451	1
SNA69641	1
FRO18919	1
SNA90258	1
ELE28573	1
ELE11375	1
DAI14125	1
FRO78087	1
ELE17451	1
GRO73461	1
DAI22896	1
SNA80192	1
SNA85662	1
SNA90258	1
DAI46755	1
FRO81176	1
ELE66810	1
DAI49199	1
DAI91535	1
GRO94758	1
ELE94711	1
DAI22177	1
ELE17451	1
SNA69641	1
DAI91535	1
GRO94758	1
GRO99222	1
FRO76833	1
FRO81176	1
SNA80192	1
DAI54690	1
ELE37798	1
GRO56989	1
ELE17451	1
GRO73461	1
DAI22896	1
GRO99222	1
SNA47306	1

In [154]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

total_purchases = 0

current_product = None
current_count = 0

for line in sys.stdin:
    splits = re.split("\t", line)
    if splits[0] == "***":
        total_purchases = int(splits[1])
        continue
    if current_product is None:
        current_product = splits[0]
        current_count = int(splits[1])
    if splits[0] != current_product:
        print(str(current_count) + "\t" + current_product + "\t" + str(1.*current_count/total_purchases))
        current_product = splits[0]
        current_count = int(splits[1])
    else:
        current_count += int(splits[1])
print(str(current_count) + "\t" + current_product + "\t" + str(1.*current_count/total_purchases))        

Overwriting reducer.py


In [155]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort | python reducer.py 

3	DAI14125	0.005253940455341506
1	DAI17810	0.0017513134851138354
2	DAI18334	0.0035026269702276708
7	DAI22177	0.012259194395796848
3	DAI22534	0.005253940455341506
10	DAI22896	0.017513134851138354
1	DAI25796	0.0017513134851138354
1	DAI32480	0.0017513134851138354
1	DAI33885	0.0017513134851138354
11	DAI35347	0.01926444833625219
1	DAI36452	0.0017513134851138354
1	DAI37607	0.0017513134851138354
3	DAI38156	0.005253940455341506
1	DAI38969	0.0017513134851138354
2	DAI44355	0.0035026269702276708
1	DAI46755	0.0017513134851138354
3	DAI47060	0.005253940455341506
3	DAI48891	0.005253940455341506
1	DAI48919	0.0017513134851138354
1	DAI49199	0.0017513134851138354
1	DAI49253	0.0017513134851138354
1	DAI50913	0.0017513134851138354
2	DAI50921	0.0035026269702276708
1	DAI52318	0.0017513134851138354
2	DAI54444	0.0035026269702276708
2	DAI54690	0.0035026269702276708
2	DAI55911	0.0035026269702276708
1	DAI58079	0.0017513134851138354
2	DAI59119	0.0035026269702276708
2	DAI59508	0.0035026269702276708
9	DAI62779	0.0157

In [159]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.3/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.3/output

rm: `/user/ubuntu/HW3/3.3/output/*': No such file or directory
packageJobJar: [/tmp/hadoop-unjar9053779728210116549/] [] /tmp/streamjob360386910639158886.jar tmpDir=null
16/01/31 19:52:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 19:52:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 19:52:49 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 19:52:49 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 19:52:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0063
16/01/31 19:52:49 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0063
16/01/31 19:52:49 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0063/
16/01/31 19:52:49 INFO mapreduce.Job: Running job: job_1453815382601_0063
16/01/31 19:52:54 INFO mapreduce.Job: Job job_1453815382601_0063 running in uber mode : false
16/01/31 19:52:54 I

In [160]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3/output/part-00000

9	DAI11153	4.725004725e-05
155	DAI11223	0.000813750813751
3	DAI11238	1.575001575e-05
1	DAI11257	5.25000525001e-06
6	DAI11261	3.15000315e-05
1	DAI11273	5.25000525001e-06
5	DAI11290	2.625002625e-05
2	DAI11299	1.05000105e-05
1	DAI11375	5.25000525001e-06
8	DAI11462	4.2000042e-05
5	DAI11541	2.625002625e-05
8	DAI11552	4.2000042e-05
25	DAI11555	0.00013125013125
1	DAI11582	5.25000525001e-06
2	DAI11613	1.05000105e-05
5	DAI11695	2.625002625e-05
1	DAI11707	5.25000525001e-06
117	DAI11778	0.000614250614251
73	DAI11927	0.00038325038325
1	DAI11946	5.25000525001e-06
1	DAI11995	5.25000525001e-06
6	DAI12036	3.15000315e-05
1	DAI12139	5.25000525001e-06
1	DAI12152	5.25000525001e-06
2	DAI12275	1.05000105e-05
16	DAI12437	8.40000840001e-05
1	DAI12448	5.25000525001e-06
1	DAI12460	5.25000525001e-06
78	DAI12521	0.0004095004095
1	DAI12535	5.25000525001e-06
3	DAI12558	1.575001575e-05
3	DAI12566	1.575001575e-05
65	DAI12680	0.00034125034125
2	DAI12683	1.05000105e-05
1	DAI12698	5.25000525001e-06
49	DAI12720	0.0002572

In [7]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    print(line.strip())

Overwriting mapper.py


In [8]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    print(line.strip())

Overwriting reducer.py


In [9]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3.sort/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.sort

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k1nr \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/output/part-00000 \
-output /user/ubuntu/HW3/3.3.sort

rm: `/user/ubuntu/HW3/3.3.sort/*': No such file or directory
rmdir: `/user/ubuntu/HW3/3.sort': No such file or directory
packageJobJar: [/tmp/hadoop-unjar1005399502662767897/] [] /tmp/streamjob3430867761648448469.jar tmpDir=null
16/02/03 17:53:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/03 17:53:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/03 17:53:14 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 17:53:14 INFO mapreduce.JobSubmitter: number of splits:2
16/02/03 17:53:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0078
16/02/03 17:53:14 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0078
16/02/03 17:53:14 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0078/
16/02/03 17:53:14 INFO mapreduce.Job: Running job: job_1453815382601_0078
16/02/03 17:53:20 INFO mapreduce.Job: Job job_14538153

In [10]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3.sort/part-00000

6667	DAI62779	0.0350017850018
3881	FRO40251	0.0203752703753
3875	ELE17451	0.0203437703438
3602	GRO73461	0.0189105189105
3044	SNA80324	0.015981015981
2851	ELE32164	0.0149677649678
2736	DAI75645	0.014364014364
2455	SNA45677	0.0128887628888
2330	FRO31317	0.0122325122325
2293	DAI85309	0.0120382620383
2292	ELE26917	0.012033012033
2233	FRO80039	0.0117232617233
2115	GRO21487	0.0111037611038
2083	SNA99873	0.0109357609358
2004	GRO59710	0.010521010521
1920	GRO71621	0.01008001008
1918	FRO85978	0.0100695100695
1840	GRO30386	0.00966000966001
1816	ELE74009	0.00953400953401
1784	GRO56726	0.00936600936601
1773	DAI63921	0.00930825930826
1756	GRO46854	0.00921900921901
1713	ELE66600	0.00899325899326
1712	DAI83733	0.00898800898801
1702	FRO32293	0.00893550893551
1697	ELE66810	0.00890925890926
1646	SNA55762	0.00864150864151
1627	DAI22177	0.00854175854176
1531	FRO78087	0.00803775803776
1516	ELE99737	0.00795900795901
1489	ELE34057	0.00781725781726
1489	GRO94758	0.00781725781726
1436	FRO35904	0.00753900753901


## HW 3.4

Write a map-reduce program to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 (i.e. product pairs need to occur together at least 100 times to be considered frequent) and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

In [181]:
!hdfs dfs -mkdir /user/ubuntu/HW3/3.4

In [182]:
#duplicating data, but it simplifies the hadoop commands
!hdfs dfs -put ./ProductPurchaseData.txt /user/ubuntu/HW3/3.4/

In [196]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

for line in sys.stdin:
    products = WORD_RE.findall(line)
    products_in_basket = len(products)
    #subtract 1 from the total because once you get to the last product
    #you've already considered everything it pairs with in this basket
    for i in range(products_in_basket-1):
        for j in range(i+1, products_in_basket):
            #Sort the two products lexically so they produce the same key
            #even if they're purchased in a different order
            if products[i] < products[j]:
                print(products[i]+":"+products[j] + "\t1")
            else:
                print(products[j]+":"+products[i] + "\t1")

Overwriting mapper.py


In [197]:
!cat ProductPurchaseDataMini.txt | python mapper.py

ELE17451:FRO11987	1
ELE89019:FRO11987	1
FRO11987:SNA90258	1
FRO11987:GRO99222	1
ELE17451:ELE89019	1
ELE17451:SNA90258	1
ELE17451:GRO99222	1
ELE89019:SNA90258	1
ELE89019:GRO99222	1
GRO99222:SNA90258	1
GRO12298:GRO99222	1
FRO12685:GRO99222	1
ELE91550:GRO99222	1
GRO99222:SNA11465	1
ELE26917:GRO99222	1
ELE52966:GRO99222	1
FRO90334:GRO99222	1
GRO99222:SNA30755	1
ELE17451:GRO99222	1
FRO84225:GRO99222	1
GRO99222:SNA80192	1
FRO12685:GRO12298	1
ELE91550:GRO12298	1
GRO12298:SNA11465	1
ELE26917:GRO12298	1
ELE52966:GRO12298	1
FRO90334:GRO12298	1
GRO12298:SNA30755	1
ELE17451:GRO12298	1
FRO84225:GRO12298	1
GRO12298:SNA80192	1
ELE91550:FRO12685	1
FRO12685:SNA11465	1
ELE26917:FRO12685	1
ELE52966:FRO12685	1
FRO12685:FRO90334	1
FRO12685:SNA30755	1
ELE17451:FRO12685	1
FRO12685:FRO84225	1
FRO12685:SNA80192	1
ELE91550:SNA11465	1
ELE26917:ELE91550	1
ELE52966:ELE91550	1
ELE91550:FRO90334	1
ELE91550:SNA30755	1
ELE17451:ELE91550	1
ELE91550:FRO84225	1
ELE91550:SNA80192	1
ELE26917:SNA11465	1
ELE52966:SNA11465	1


In [198]:
%%writefile reducer.py
#!/usr/bin/python

import sys,re

current_pair = None
current_count = 0

for line in sys.stdin:
    splits = [x.strip() for x in re.split("\t", line)]
    if current_pair == None:
        current_pair = splits[0]
        current_count = int(splits[1])
    if splits[0] != current_pair:
        if current_count >= 100:
            print(str(current_count) + "\t" + current_pair)
        current_pair = splits[0]
        current_count = int(splits[1])
    else:
        current_count += int(splits[1])
if current_count >= 100:
    print(str(current_count) + "\t" + current_pair)

Overwriting reducer.py


In [199]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort | python reducer.py

In [200]:
!hdfs dfs -rm /user/ubuntu/HW3/3.4/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.4/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.4/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.4/output

16/01/31 21:10:30 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.4/output/_SUCCESS
16/01/31 21:10:30 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.4/output/part-00000
packageJobJar: [/tmp/hadoop-unjar400689311636777875/] [] /tmp/streamjob2455226103313207350.jar tmpDir=null
16/01/31 21:10:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 21:10:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 21:10:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 21:10:35 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 21:10:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0066
16/01/31 21:10:35 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0066
16/01/31 21:10:

In [201]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4/output/part-00000 | wc -l

1334


In [202]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4/output/part-00000 | head -n50

106	DAI16732:FRO78087
102	DAI18527:SNA44451
127	DAI22177:DAI31081
382	DAI22177:DAI62779
136	DAI22177:DAI63921
123	DAI22177:DAI75645
126	DAI22177:DAI83733
172	DAI22177:DAI85309
203	DAI22177:ELE17451
134	DAI22177:ELE26917
155	DAI22177:ELE32164
107	DAI22177:ELE34057
134	DAI22177:ELE56788
101	DAI22177:ELE66600
105	DAI22177:ELE66810
108	DAI22177:ELE74009
150	DAI22177:ELE91337
160	DAI22177:FRO31317
128	DAI22177:FRO32293
181	DAI22177:FRO40251
130	DAI22177:FRO66272
107	DAI22177:FRO78087
152	DAI22177:FRO80039
156	DAI22177:FRO85978
122	DAI22177:GRO21487
106	DAI22177:GRO30386
160	DAI22177:GRO46854
120	DAI22177:GRO59710
132	DAI22177:GRO71621
248	DAI22177:GRO73461
145	DAI22177:SNA45677
102	DAI22177:SNA55762
140	DAI22177:SNA80324
148	DAI22177:SNA99873
151	DAI22240:DAI62779
105	DAI22240:ELE17451
219	DAI22240:ELE37048
103	DAI22240:ELE74482
112	DAI22240:FRO40251
123	DAI22534:DAI62779
297	DAI22896:DAI62779
215	DAI22896:DAI75645
193	DAI22896:ELE17451
107	DAI22896:ELE32164
165	DAI22896:ELE74009
167	DAI228

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2.

In [8]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

total_baskets = 0

WORD_RE = re.compile(r"[\w']+")

for line in sys.stdin:
    total_baskets += 1
    products = WORD_RE.findall(line)
    products_in_basket = len(products)
    #subtract 1 from the total because once you get to the last product
    #you've already considered everything it pairs with in this basket
    for i in range(products_in_basket-1):
        for j in range(i+1, products_in_basket):
            #Sort the two products lexically so they produce the same key
            #even if they're purchased in a different order
            if products[i] < products[j]:
                print(products[i]+":"+products[j] + "\t1")
            else:
                print(products[j]+":"+products[i] + "\t1")    
                
print("***\t" + str(total_baskets))

Overwriting mapper.py


In [9]:
!cat ProductPurchaseDataMini.txt | python mapper.py

ELE17451:FRO11987	1
ELE89019:FRO11987	1
FRO11987:SNA90258	1
FRO11987:GRO99222	1
ELE17451:ELE89019	1
ELE17451:SNA90258	1
ELE17451:GRO99222	1
ELE89019:SNA90258	1
ELE89019:GRO99222	1
GRO99222:SNA90258	1
GRO12298:GRO99222	1
FRO12685:GRO99222	1
ELE91550:GRO99222	1
GRO99222:SNA11465	1
ELE26917:GRO99222	1
ELE52966:GRO99222	1
FRO90334:GRO99222	1
GRO99222:SNA30755	1
ELE17451:GRO99222	1
FRO84225:GRO99222	1
GRO99222:SNA80192	1
FRO12685:GRO12298	1
ELE91550:GRO12298	1
GRO12298:SNA11465	1
ELE26917:GRO12298	1
ELE52966:GRO12298	1
FRO90334:GRO12298	1
GRO12298:SNA30755	1
ELE17451:GRO12298	1
FRO84225:GRO12298	1
GRO12298:SNA80192	1
ELE91550:FRO12685	1
FRO12685:SNA11465	1
ELE26917:FRO12685	1
ELE52966:FRO12685	1
FRO12685:FRO90334	1
FRO12685:SNA30755	1
ELE17451:FRO12685	1
FRO12685:FRO84225	1
FRO12685:SNA80192	1
ELE91550:SNA11465	1
ELE26917:ELE91550	1
ELE52966:ELE91550	1
ELE91550:FRO90334	1
ELE91550:SNA30755	1
ELE17451:ELE91550	1
ELE91550:FRO84225	1
ELE91550:SNA80192	1
ELE26917:SNA11465	1
ELE52966:SNA11465	1


In [7]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

total_baskets = 0

current_pair = "***"
current_count = 0

frequent = 100

for line in sys.stdin:
    splits = [x.strip() for x in re.split("\t", line)]
    if splits[0] == "***":
        total_baskets = int(splits[1])
        continue
    if splits[0] != current_pair:
        if current_count >= frequent:
            print(str(current_count) + "\t" + str(1.*current_count/total_baskets) + "\t" + current_pair)
        current_pair = splits[0]
        current_count = int(splits[1])
    else:
        current_count += int(splits[1])
if current_count >= frequent:
    print(str(current_count) + "\t" + str(1.*current_count/total_baskets) + "\t" + current_pair)    

Overwriting reducer.py


In [6]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort | python reducer.py

10	0.2	DAI35347:ELE17451
11	0.22	ELE17451:GRO73461
15	0.3	ELE17451:GRO99222
10	0.2	ELE26917:GRO99222


In [10]:
!hdfs dfs -rm /user/ubuntu/HW3/3.4/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.4/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.4/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.4/output

16/02/01 01:28:00 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.4/output/_SUCCESS
16/02/01 01:28:00 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.4/output/part-00000
packageJobJar: [/tmp/hadoop-unjar4255667787579131764/] [] /tmp/streamjob9050589593524720090.jar tmpDir=null
16/02/01 01:28:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 01:28:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 01:28:05 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 01:28:05 INFO mapreduce.JobSubmitter: number of splits:2
16/02/01 01:28:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0068
16/02/01 01:28:05 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0068
16/02/01 01:28

In [11]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4/output/part-00000 | wc -l

1334


In [12]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4/output/part-00000 | head -n50

106	0.00706478272461	DAI16732:FRO78087
102	0.00679818715009	DAI18527:SNA44451
127	0.0084644094908	DAI22177:DAI31081
382	0.025459877366	DAI22177:DAI62779
136	0.00906424953346	DAI22177:DAI63921
123	0.00819781391629	DAI22177:DAI75645
126	0.00839776059717	DAI22177:DAI83733
172	0.0114636097041	DAI22177:DAI85309
203	0.0135297254066	DAI22177:ELE17451
134	0.0089309517462	DAI22177:ELE26917
155	0.0103305785124	DAI22177:ELE32164
107	0.00713143161824	DAI22177:ELE34057
134	0.0089309517462	DAI22177:ELE56788
101	0.00673153825646	DAI22177:ELE66600
105	0.00699813383098	DAI22177:ELE66810
108	0.00719808051186	DAI22177:ELE74009
150	0.00999733404425	DAI22177:ELE91337
160	0.0106638229805	DAI22177:FRO31317
128	0.00853105838443	DAI22177:FRO32293
181	0.0120634497467	DAI22177:FRO40251
130	0.00866435617169	DAI22177:FRO66272
107	0.00713143161824	DAI22177:FRO78087
152	0.0101306318315	DAI22177:FRO80039
156	0.010397227406	DAI22177:FRO85978
122	0.00813116502266	DAI22177:GRO21487
106	0.00706478272461	DAI22177:GRO30386

In [13]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    print(line.strip())

Overwriting mapper.py


In [14]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

count = 50

for line in sys.stdin:
    print(line.strip())
    count -= 1
    if count <= 0:
        break

Overwriting reducer.py


In [16]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4/output/part-00000 | head -n50 | python mapper.py | sort -k1nr | python reducer.py

cat: Unable to write to output stream.
382	0.025459877366	DAI22177:DAI62779
297	0.0197947214076	DAI22896:DAI62779
248	0.0165289256198	DAI22177:GRO73461
219	0.0145961077046	DAI22240:ELE37048
215	0.0143295121301	DAI22896:DAI75645
203	0.0135297254066	DAI22177:ELE17451
193	0.0128632364703	DAI22896:ELE17451
181	0.0120634497467	DAI22177:FRO40251
172	0.0114636097041	DAI22177:DAI85309
167	0.0111303652359	DAI22896:FRO31317
165	0.0109970674487	DAI22896:ELE74009
160	0.0106638229805	DAI22177:FRO31317
160	0.0106638229805	DAI22177:GRO46854
156	0.010397227406	DAI22177:FRO85978
155	0.0103305785124	DAI22177:ELE32164
154	0.0102639296188	DAI22896:FRO40251
152	0.0101306318315	DAI22177:FRO80039
151	0.0100639829379	DAI22240:DAI62779
150	0.00999733404425	DAI22177:ELE91337
148	0.009864036257	DAI22177:SNA99873
145	0.00966408957611	DAI22177:SNA45677
140	0.00933084510797	DAI22177:SNA80324
136	0.00906424953346	DAI22177:DAI63921
134	0.0089309517462	DAI22177:ELE26917
134	0.0089309517462	DAI22177:ELE56788
132	0.0087

In [17]:
!hdfs dfs -rm /user/ubuntu/HW3/3.4.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.4.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k1nr \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.4/output/part-00000 \
-output /user/ubuntu/HW3/3.4.2/output

rm: `/user/ubuntu/HW3/3.4.2/output/*': No such file or directory
rmdir: `/user/ubuntu/HW3/3.4.2/output': No such file or directory
packageJobJar: [/tmp/hadoop-unjar5143549329236513740/] [] /tmp/streamjob5154196380315583025.jar tmpDir=null
16/02/01 01:39:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 01:39:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 01:39:06 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 01:39:06 INFO mapreduce.JobSubmitter: number of splits:2
16/02/01 01:39:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0069
16/02/01 01:39:06 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0069
16/02/01 01:39:06 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0069/
16/02/01 01:39:06 INFO mapreduce.Job: Running job: job_1453815382601_0069
16/02/01 01:39:12 INFO mapreduce.Job: Job jo

In [18]:
!hdfs dfs -cat /user/ubuntu/HW3/3.4.2/output/part-00000

1592	0.106105038656	DAI62779:ELE17451
1412	0.0941082378033	FRO40251:SNA80324
1254	0.08357771261	DAI75645:FRO40251
1213	0.0808451079712	FRO40251:GRO85051
1139	0.0759130898427	DAI62779:GRO73461
1130	0.0753132498001	DAI75645:SNA80324
1070	0.0713143161824	DAI62779:FRO40251
923	0.061516928819	DAI62779:SNA80324
918	0.0611836843508	DAI62779:DAI85309
911	0.0607171420954	ELE32164:GRO59710
882	0.0587843241802	FRO40251:GRO73461
882	0.0587843241802	DAI62779:DAI75645
877	0.0584510797121	DAI62779:ELE92920
835	0.0556518261797	FRO40251:FRO92469
832	0.0554518794988	DAI62779:ELE32164
712	0.0474540122634	DAI75645:GRO73461
711	0.0473873633698	DAI43223:ELE32164
709	0.0472540655825	DAI62779:GRO30386
697	0.046454278859	ELE17451:FRO40251
659	0.0439216209011	DAI85309:ELE99737
650	0.0433217808584	DAI62779:ELE26917
631	0.0420554518795	GRO21487:GRO73461
604	0.0402559317515	DAI62779:SNA45677
597	0.0397893894961	ELE17451:SNA80324
595	0.0396560917089	DAI62779:GRO71621
593	0.0395227939216	DAI62779:SNA55762
586	0.0390

## HW 3.5
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

In [ ]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

total_baskets = 0
stripes = {}

WORD_RE = re.compile(r"[\w']+")

for line in sys.stdin:
    total_baskets += 1
    products = WORD_RE.findall(line)
    products_in_basket = len(products)
    #subtract 1 from the total because once you get to the last product
    #you've already considered everything it pairs with in this basket
    for i in range(products_in_basket-1):
        for j in range(i+1, products_in_basket):
            #Sort the two products lexically so they produce the same key
            #even if they're purchased in a different order
            if products[i] < products[j]:
                product_one = products[i]
                product_two = products[j]
            else:
                product_one = products[j]
                product_two = products[i]

            if product_one in stripes:
                stripes[product_one].append(product_two)
            else:
                stripes[product_one = []
                stripes[product_one.append(product_two)

for               
print("***\t" + str(total_baskets))

In [6]:
testDict = {}
testDict['a'] = [1,2,3,4,5]
testDict['b'] = 6
testDict['c'] = 7
testDict
for key, value in testDict.items():
    print(key, value)

a [1, 2, 3, 4, 5]
c 7
b 6
